# Configurations

Configurations of components save relevant data they may have, so they can be loaded. They can represent a single Component, or a Component Tree, with child_components.

All components have 3 dictionaries:

    - Input, for the input passed for the component
    - Values, for values the component has exposed and represent its current state
    - Output, for values the component is meant to expose after it is "used"

And have list of strings designated as "notes", in which notes can be added to the component to give relevant context in later uses

An example is a model which was loaded and had its parameters perturbed, we may want to remember where the original model came from and the parameters used to perturb its parameters, and so we can save that in the notes of it

## Experiments Configurations

Experiments are meant to be defined as configurations, either as a dictionary defined in python or JSON.

Usually the only values defined are the component used and non default values used for their input

### PPO Training Configuration example

For example, let's look at the PPO configuration done with the hyperparameters presented by the trained model for the CartPole Environment

In [ ]:
from automl.utils.class_util import make_classes_in_collection_strings

#from automl.base_configurations.environment.cart_pole.ppo_sb3 import config_dict as base_config_dict
from automl.base_configurations.environment.cart_pole.dqn_sb3 import config_dict as base_config_dict



non_str_base_config_dict = base_config_dict()

str_base_config_dict = make_classes_in_collection_strings(non_str_base_config_dict)



In [ ]:
import json

old_json_string_of_component = json.dumps(str_base_config_dict, indent=4)


print(old_json_string_of_component) # this is to print the dictionary in readable indentation

## Altering Experiments

It is easy to load a defined experiment and change parts of it as we see fit

The only labour we have to do is change the specific value 

# Instantiating and saving a component

## Insantiating a component from a dict / json string

In [ ]:
from automl.utils.json_utils.json_component_utils import gen_component_from

ppo_experiment = gen_component_from(non_str_base_config_dict)

In [ ]:
print(ppo_experiment.name)
print(ppo_experiment.input)

In [ ]:
ppo_experiment.pass_input({"base_directory" : "example_pipeline"})

ppo_experiment.proccess_input_if_not_proccesd() # we proccess the input as to change the component

## Making the component a json string

In [ ]:
from automl.utils.json_utils.json_component_utils import json_string_of_component

new_json_string_of_component = json_string_of_component(ppo_experiment, 
                                                        ignore_defaults=False, 
                                                        save_exposed_values=True, 
                                                        respect_ignore_order=False)


In [ ]:
print(new_json_string_of_component)

Note that a lot of components are yet to be initialized, this is because all processing is done at the last moment possible, and those components were not yet needed.

# Localizations

This represents a localization of a component in a component tree, made by following the source component and its children

Total localizations are given by lists in which each element is a step of going through the current component to another

Localizations are usually stored in a component, and so there are two different types of them:

"absolute", which goes from the source component
"relative", which goes from the current component

Each element of a localization can be:

- An integer, to represent jump from the current component to some component at that index in the child components
- A string, to represent jump from the current component to some component with that name in the child components or their own child components

String can represent other "look" operations:
- "\_\_up\_\_", to represent a step to the parent component (this should only be used in a relative localization)
- A tuple (operation_str, {parameters}) with some special operation:
- ("\_\_get\_by\_name\_\_", { name_of_component })
- ("\_\_get\_exposed\_value\_\_", {value_localization})
- ("\_\_get\_input\_value\_\_", {value_localization})



For example, we can get the "capacity" value of the learning rate with:

In [ ]:
from automl.component import Component
from automl.core.localizations import get_component_by_localization

# get optimizer component
rl_trainer : Component = get_component_by_localization(ppo_experiment, 
[
    ["__get_by_name__", {"name_of_component" : "RLTrainerComponent"}],
])

print(f"Optimizer component: {rl_trainer}") # we got the optimizer component

print(f"Limiter of total steps: {rl_trainer.get_input_value("limit_total_steps")}")

We can also totaly get using only the localization

This format is useful when specifying localizations in the configuration texts

In [ ]:
from automl.core.localizations import get_component_by_localization

# we can also get it directly
limit_total_steps = get_component_by_localization(ppo_experiment, 
[
    ["__get_by_name__", {"name_of_component" : "RLTrainerComponent"}],
    ["__get_input_value__", {"value_localization" : ["limit_total_steps"]}]
])

print(f"Limiter of total steps: {limit_total_steps}")